<a href="https://colab.research.google.com/github/rmlessa/projeto_final/blob/main/projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perguntas de Negócio

1. A renda média do bairro que recebe o abastecimento está diretamente ligada a qualidade da água fornecida?

2. A densidade populacional por bairro está relacionada com a qualidade da água fornecida?

3. Quais bairros de Manaus necessitam de tratamento de água adicional?

In [1]:
#instala o pyspark
!pip install pyspark

     |████████████████████████████████| 204.8MB 74kB/s 
     |████████████████████████████████| 204kB 21.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=d578f0126b224a2375bdf0b3101df2a3df3f7c83d5aa2978ecffb95b11e611d3
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [38]:
#importa bibliotecas que serão utilizadas
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id #incrementa indice dos dataframes pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim

In [3]:
#monta o google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#cria sessão spark no colab para realizar a extração dos dados dos arquivos countries.csv e flags.csv
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("etl") \
        .getOrCreate()

In [5]:
analise_agua_2020 = '/content/drive/MyDrive/01 - AQUISIÇÃO DE DADOS/Projeto_Final/2amostras_de_vigilancia_parametros_basicos_anual_2020.csv'

In [6]:
#dataframe pyspark para receber dados do arquivo csv
session_2020 = scSpark.read.csv(analise_agua_2020, header=True, encoding="ISO-8859-1" ,sep=";").cache() #armazena o resultado em cache para melhor desempenho
print('Total de Registros em 2020.csv = {}'.format(session_2020.count()))
session_2020.show()

Total de Registros em 2020.csv = 2125798
+-----------------+--------+--------------------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|      NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição|

In [32]:
#remover espaço em branco do valor das colunas
session_2020 = session_2020.withColumn("REGIAO_GEOGRAFICA", trim(col="REGIAO_GEOGRAFICA"))
session_2020 = session_2020.withColumn("NOME_MUNICIPIO", trim(col="NOME_MUNICIPIO"))
session_2020 = session_2020.withColumn("AREA", trim(col="AREA"))
session_2020.show()

+-----------------+--------+--------------------+------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|    NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição| Nome da Instituição|    CNPJ_INSTITUICAO|Nom

In [33]:
#transforma em tabelas para recuperar os dados da região norte, principalmente Manaus
session_2020.createOrReplaceTempView("vw_dados_regiao_norte_2020")

#reduz os dados da tabela para trabalharmos apenas com a região norte
dados_regiao_norte_2020 =  scSpark.sql("SELECT * from vw_dados_regiao_norte_2020 where regiao_geografica = 'NORTE' and SIGLA_UF = 'AM' AND NOME_MUNICIPIO = 'MANAUS' ORDER BY SIGLA_UF")
print('Total de dados_regiao_norte_2020 = {}'.format(dados_regiao_norte_2020.count()))
dados_regiao_norte_2020.show()

Total de dados_regiao_norte_2020 = 16277
+-----------------+--------+--------------------+--------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------+---------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição| Nome da Instituição|    CNPJ_INSTI

In [37]:
#após a redução para dados da região norte, analisei que colunas seriam necessárias para trabalharmos
#comecei identificando que colunas estão COM DADOS NULOS PARA REGIÃO NORTE. Identifiquei latitude e longitude
col_latitude = scSpark.sql("SELECT latitude from vw_dados_regiao_norte_2020 where REGIAO_GEOGRAFICA = 'norte' and latitude is not null")
col_latitude.show()
col_longitude = scSpark.sql("SELECT longitude from vw_dados_regiao_norte_2020 where REGIAO_GEOGRAFICA = 'norte' and longitude is not null")
col_longitude.show()

tipos_analises = scSpark.sql("SELECT * from vw_dados_regiao_norte_2020 where REGIAO_GEOGRAFICA = 'NORTE' AND NOME_MUNICIPIO = 'MANAUS' and (parametro like '%Coli%' or parametro like '%coli%')" )
print('Total de Linhas = {}'.format(tipos_analises.count()))
tipos_analises.show()



tipos_analises_bairros = scSpark.sql("SELECT * from vw_dados_regiao_norte_2020 where REGIAO_GEOGRAFICA = 'NORTE' AND NOME_MUNICIPIO = 'MANAUS' and (parametro like '%Coli%' or parametro like '%coli%') and AREA = 'JORGE TEIXEIRA'" )
print('Total de Linhas = {}'.format(tipos_analises_bairros.count()))
tipos_analises_bairros.show()

#acredito que essas colunas podem ser removidas, uma vez que não possuem dados. Outra coluna que acho que pode ser removida é a coluna
#região_geográfica. Como filtramos pela região norte, ela acaba não se fazendo necessária.



+--------+
|latitude|
+--------+
+--------+

+---------+
|longitude|
+---------+
+---------+

Total de Linhas = 5671
+-----------------+--------+--------------------+--------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+--------+---------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO|